In [1]:
import re, time, csv, requests, pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm import tqdm


In [2]:
BASE_URL  = "https://force-n.sn"
HEADERS   = {"User-Agent": "Mozilla/5.0"}
DELAY_SEC = 1                              

In [3]:
def get_soup(url: str) -> BeautifulSoup:
    res = requests.get(url, headers=HEADERS, timeout=15)
    res.raise_for_status()
    return BeautifulSoup(res.content, "html.parser")


In [4]:
def discover_menu_links() -> dict:
    """Récupère automatiquement les entrées du menu principal."""
    soup = get_soup(BASE_URL)
    links = {}
    nav = soup.find("nav", id=re.compile("forcen-main-menu"))
    for a in nav.find_all("a", href=True):
        label = a.get_text(strip=True) or a["href"]
        href  = urljoin(BASE_URL, a["href"])
        if href.startswith(BASE_URL):
            links[label] = href
    return links


In [5]:
def scrape_page(url: str) -> dict:
    """Renvoie un dict {url, titres_concats, contenu} pour une page donnée."""
    soup = get_soup(url)
    titres   = [h.get_text(strip=True) for h in soup.find_all(["h1", "h2", "h3"])]
    paragraphes = [
        p.get_text(strip=True)
        for p in soup.find_all("p")
        if len(p.get_text(strip=True)) > 30
    ]
    return {
        "url": url,
        "titres": " | ".join(titres),
        "contenu": "\n".join(paragraphes)
    }

In [6]:

def main():
    menu_links = discover_menu_links()
    print(" Pages détectées :", list(menu_links.keys()))

    rows = []
    for titre, url in tqdm(menu_links.items(), desc="Scraping"):
        try:
            info = scrape_page(url)
            info["page"] = titre            
            rows.append(info)
            time.sleep(DELAY_SEC)
        except Exception as e:
            print(f" Échec {url} → {e}")

    out_file = "connaissances_force-n.csv"
    pd.DataFrame(rows).to_csv(out_file, index=False, quoting=csv.QUOTE_ALL)
    print(f" Fichier généré : {out_file} ({len(rows)} pages)")

if __name__ == "__main__":
    main()


 Pages détectées : ['A propos', 'Formations', 'SIGUI', 'Parcours initiatique', 'Entrepreneuriat', 'Services', 'Communauté', 'Promotion des sciences', 'Actualités']


Scraping: 100%|██████████| 9/9 [00:12<00:00,  1.39s/it]

 Fichier généré : connaissances_force-n.csv (9 pages)
